In [14]:
from collections import defaultdict, Counter

In [15]:
import pandas as pd
import nltk
# import sklearn as sk
import gensim.models
# from nltk.corpus import stopwords
# import string
import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
# from ast import literal_eval
# from predictor import *
import predictor
# sns.set()

clean = pd.read_csv("/Users/aditya/Documents/GitHub/NLP-for-motivation-lab/CleanData(v. analysis).csv")

colname = ['id', 'condition', 'wish', 'outcome', 'outcome2', 'useful', 'mins', 'other', 'reuse', 'clarity', 'benefit', 'interest', 'learn']
clean = clean.iloc[:256,:13]
clean.columns= colname
clean.fillna("", inplace=True)

In [16]:
clean['all'] = clean.wish + " " + clean.outcome + " " + clean.outcome2
clean.drop(columns = ['wish', 'outcome', 'outcome2'], inplace=True)

# clean['all'] = clean['all'].apply(remove_stop)
# model = gensim.models.Word2Vec(clean['all'], min_count = 2)
# model.most_similar('happy')

# clean = pd.get_dummies(clean, columns = ['useful','mins','other','reuse', 'clarity', 'benefit', 'interest', 'learn'])

coders = pd.read_csv('/Users/aditya/Documents/GitHub/NLP-for-motivation-lab/MCII coding replication - coding file(v. analysis).csv')
colname = ['id', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 's1', 's2', 's3', 's4', 's5', 's6']
coders = coders.iloc[:256,:13]
coders.columns= colname
coders.fillna(1, inplace=True)
coders['d'] = round(coders.loc[:,'d1':'d6'].mean(axis=1))
coders['s'] = round(coders.loc[:,'s1':'s6'].mean(axis=1))
clean = pd.concat([clean, coders], join='outer', axis=1)
clean.head()
clean.head()

,id,condition,useful,mins,other,reuse,clarity,benefit,interest,learn,...,d5,d6,s1,s2,s3,s4,s5,s6,d,s
0,558af269fdf99b7dfef4d7bd,MC,7.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,...,2.0,5.0,2.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0
1,55b1add5fdf99b6731f04c5a,MC,1.0,1.0,2.0,1.0,3.0,1.0,2.0,2.0,...,5.0,5.0,4.0,3.0,5.0,4.0,3.0,5.0,5.0,4.0
2,5626a6cfddff3c0011bcd3d8,I,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,3.0,5.0,4.0,3.0,5.0,4.0,4.0,4.0
3,56a5da6181a8ca000d3e7f40,MC,4.0,1.0,4.0,3.0,2.0,3.0,5.0,4.0,...,5.0,5.0,3.0,3.0,4.0,3.0,3.0,4.0,4.0,3.0
4,56df2ea2452e1300056bc7df,MC,3.0,1.0,4.0,4.0,3.0,3.0,4.0,3.0,...,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0


In [17]:
# for each_col in clean.loc[:,'useful':'learn']:
#     print(each_col)
#     plt.hist(clean[each_col])
#     print(clean[each_col].value_counts())
#     plt.show()

In [18]:
history_scores = defaultdict(dict)

In [19]:
def variation(kind, solver):
    history_scores[f'{solver}'][f'{kind}'] = {}
    for each_col in clean.loc[:,'useful':'learn']:
        mypredictor = predictor.predictor(clean, 'all', kind, solver)
        print("\n" + each_col)
        mypredictor.train_on(each_col)
        history_scores[f'{mypredictor.solver}'][f'{mypredictor.kind}'][f'{each_col}'] = \
        round(mypredictor.test(), 3)

In [20]:
variation('l2', 'liblinear')


useful
{'fit_time': array([0.01347804, 0.01676607, 0.01780295]), 'score_time': array([0.00120687, 0.00108099, 0.00123525]), 'test_score': array([0.62962963, 0.65925926, 0.59701493]), 'train_score': array([0.97769517, 0.94423792, 0.93333333])}
0.7361111111111112

mins
{'fit_time': array([0.01439691, 0.01247191, 0.00864816]), 'score_time': array([0.00142908, 0.00069594, 0.00068879]), 'test_score': array([0.7345679 , 0.80246914, 0.83333333]), 'train_score': array([0.97530864, 0.95987654, 0.92592593])}
0.8953488372093024

other
{'fit_time': array([0.01132703, 0.01012588, 0.01739001]), 'score_time': array([0.000705  , 0.00097609, 0.00105095]), 'test_score': array([0.66115702, 0.57024793, 0.60833333]), 'train_score': array([0.94605809, 0.95020747, 0.9214876 ])}
0.5846153846153846

reuse
{'fit_time': array([0.01792908, 0.0149231 , 0.0114181 ]), 'score_time': array([0.00139189, 0.00109911, 0.00151515]), 'test_score': array([0.63309353, 0.58992806, 0.65217391]), 'train_score': array([0.9458483

In [21]:
variation('l1', 'liblinear')


useful
{'fit_time': array([0.01677513, 0.00918293, 0.00948238]), 'score_time': array([0.00088   , 0.00071216, 0.00076389]), 'test_score': array([0.52592593, 0.4       , 0.44029851]), 'train_score': array([0.53159851, 0.59107807, 0.58518519])}
0.6111111111111112

mins
{'fit_time': array([0.00826287, 0.01328182, 0.01031804]), 'score_time': array([0.00072718, 0.00845718, 0.00072503]), 'test_score': array([0.5617284 , 0.61111111, 0.51234568]), 'train_score': array([0.68518519, 0.66666667, 0.7345679 ])}
0.6627906976744186

other
{'fit_time': array([0.00885296, 0.00905585, 0.00804019]), 'score_time': array([0.00065827, 0.00073028, 0.00112486]), 'test_score': array([0.38016529, 0.3553719 , 0.35833333]), 'train_score': array([0.45643154, 0.42738589, 0.45041322])}
0.35384615384615387

reuse
{'fit_time': array([0.01150298, 0.00892472, 0.00892305]), 'score_time': array([0.00067019, 0.00081038, 0.00081491]), 'test_score': array([0.38129496, 0.32374101, 0.43478261]), 'train_score': array([0.490974

In [22]:
variation('l2', 'lbfgs')


useful
{'fit_time': array([0.54537702, 0.6873951 , 0.55810022]), 'score_time': array([0.0006361 , 0.00156188, 0.00120211]), 'test_score': array([0.6       , 0.66666667, 0.67164179]), 'train_score': array([0.9739777 , 0.94423792, 0.95925926])}
0.6944444444444444

mins
{'fit_time': array([0.25978518, 0.32074404, 0.206043  ]), 'score_time': array([0.00071001, 0.00071788, 0.00061178]), 'test_score': array([0.89506173, 0.84567901, 0.84567901]), 'train_score': array([0.97530864, 0.99074074, 0.97839506])}
0.8604651162790697

other
{'fit_time': array([0.31186414, 0.83614421, 0.58006811]), 'score_time': array([0.0012629 , 0.00099683, 0.00128388]), 'test_score': array([0.55371901, 0.63636364, 0.61666667]), 'train_score': array([0.96680498, 0.96680498, 0.94214876])}
0.6153846153846154

reuse
{'fit_time': array([0.76118994, 0.87163424, 0.47901678]), 'score_time': array([0.00082183, 0.00065589, 0.00084901]), 'test_score': array([0.61870504, 0.68345324, 0.55797101]), 'train_score': array([0.9566787

In [23]:
variation('l2', 'newton-cg')


useful
{'fit_time': array([0.10228872, 0.11856484, 0.12896729]), 'score_time': array([0.00062013, 0.00089693, 0.00113487]), 'test_score': array([0.68148148, 0.65925926, 0.58955224]), 'train_score': array([0.94795539, 0.96282528, 0.96296296])}
0.6944444444444444

mins
{'fit_time': array([0.07064486, 0.07413006, 0.06009412]), 'score_time': array([0.00091815, 0.00094199, 0.00083995]), 'test_score': array([0.82098765, 0.78395062, 0.7345679 ]), 'train_score': array([0.97222222, 0.95987654, 0.97530864])}
0.8604651162790697

other
{'fit_time': array([0.09083509, 0.07690477, 0.10461736]), 'score_time': array([0.00118017, 0.00077605, 0.00063586]), 'test_score': array([0.66115702, 0.63636364, 0.575     ]), 'train_score': array([0.96680498, 0.94190871, 0.95454545])}
0.676923076923077

reuse
{'fit_time': array([0.09430003, 0.22779202, 0.10845709]), 'score_time': array([0.00227308, 0.00104189, 0.00077391]), 'test_score': array([0.71942446, 0.65467626, 0.67391304]), 'train_score': array([0.96389892

In [24]:
history_scores

defaultdict(dict,
            {'liblinear': {'l2': {'useful': 0.736,
               'mins': 0.895,
               'other': 0.585,
               'reuse': 0.784,
               'clarity': 0.612,
               'benefit': 0.643,
               'interest': 0.718,
               'learn': 0.58},
              'l1': {'useful': 0.611,
               'mins': 0.663,
               'other': 0.354,
               'reuse': 0.514,
               'clarity': 0.522,
               'benefit': 0.457,
               'interest': 0.628,
               'learn': 0.319}},
             'lbfgs': {'l2': {'useful': 0.694,
               'mins': 0.86,
               'other': 0.615,
               'reuse': 0.649,
               'clarity': 0.642,
               'benefit': 0.714,
               'interest': 0.692,
               'learn': 0.71}},
             'newton-cg': {'l2': {'useful': 0.694,
               'mins': 0.86,
               'other': 0.677,
               'reuse': 0.676,
               'clarity': 0.642,


In [25]:
#checking predictor performance for specificity

mypredictor_coding = predictor.predictor(clean, 'all', k_neighbors = 3)
mypredictor_coding.train_on('s')
mypredictor_coding.test()

{'fit_time': array([0.66337395, 0.651057  , 0.63933992]), 'score_time': array([0.00099802, 0.00068092, 0.00115108]), 'test_score': array([0.78612717, 0.83236994, 0.8372093 ]), 'train_score': array([0.92753623, 0.94782609, 0.93352601])}
0.8369565217391305


0.8369565217391305

In [26]:
mypredictor_coding.predict("Get extra hours at work so I can be able to pay for things my family needs I would be able to take care of my family and what they need or want If I was able to fulfill this goal, I would have extra hours to able to bring more money home to my family and take care of the little things that we need and want. We could be able to go on vacation and do whatever extra stuff that we want.")

TypeError: expected dimension <= 2 array or matrix

In [27]:
pd.DataFrame(mypredictor_coding.model.coef_[0][:-1], 
             mypredictor_coding.vectorizer.get_feature_names(), 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

,coef
best outcome,1.121172
no,1.068561
outcome,1.054009
nice,1.037671
best,0.968832
...,...
eating,-0.513334
satisfied,-0.584102
pizza,-0.641077
very excited,-0.652505


In [28]:
#checking predictor performance for directness

mypredictor_coding = predictor.predictor(clean, 'all')
mypredictor_coding.train_on('d')
mypredictor_coding.test()

{'fit_time': array([0.67229128, 0.45767975, 0.66482997]), 'score_time': array([0.00058866, 0.00114202, 0.00134015]), 'test_score': array([0.78378378, 0.76870748, 0.81632653]), 'train_score': array([0.93877551, 0.93898305, 0.94237288])}
0.8076923076923077


0.8076923076923077

In [29]:
mypredictor_coding.predict("Get extra hours at work so I can be able to pay for things my family needs I would be able to take care of my family and what they need or want If I was able to fulfill this goal, I would have extra hours to able to bring more money home to my family and take care of the little things that we need and want. We could be able to go on vacation and do whatever extra stuff that we want.")

TypeError: expected dimension <= 2 array or matrix

In [30]:
pd.DataFrame(mypredictor_coding.model.coef_[0][:-1], 
             mypredictor_coding.vectorizer.get_feature_names(), 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

,coef
nice,1.152689
no,1.073091
you,1.064943
car,1.011018
way,0.787090
...,...
to,-0.563204
workout,-0.571160
will,-0.753253
feel,-0.779748


In [31]:
#effect of imbalance? 
#interpreting coeff? 
#useful exercise? What can be done? 
#how better approximate coders? 

In [32]:
coder1_specific_d = clean[['all','d1','d2','d3']][:128]
coder1_specific_d['d'] = round(coder1_specific_d[coder1_specific_d.columns[-3:]].mean(axis=1))
coder2_specific_d = clean[['all','d1','d2','d3']][128:]
coder2_specific_d['d'] = round(coder2_specific_d[coder2_specific_d.columns[-3:]].mean(axis=1))
coder3_specific_d = clean[['all','d4','d5','d6']][:128]
coder3_specific_d['d'] = round(coder3_specific_d[coder3_specific_d.columns[-3:]].mean(axis=1))
coder4_specific_d = clean[['all','d4','d5','d6']][128:]
coder4_specific_d['d'] = round(coder4_specific_d[coder4_specific_d.columns[-3:]].mean(axis=1))

In [33]:
coder1_specific_d_predictor = predictor.predictor(coder1_specific_d, 'all')
coder1_specific_d_predictor.train_on('d')
coder1_specific_d_predictor.test()

{'fit_time': array([0.28684092, 0.37595987, 0.25536489]), 'score_time': array([0.00053596, 0.00070596, 0.00483608]), 'test_score': array([0.69230769, 0.78461538, 0.61538462]), 'train_score': array([0.95384615, 0.96153846, 0.95384615])}
0.8


0.8

In [34]:
coder2_specific_d_predictor = predictor.predictor(coder2_specific_d, 'all')
coder2_specific_d_predictor.train_on('d')
coder2_specific_d_predictor.test()

{'fit_time': array([0.20186615, 0.14877892, 0.23002601]), 'score_time': array([0.00054669, 0.00054693, 0.00083613]), 'test_score': array([0.57142857, 0.55102041, 0.60416667]), 'train_score': array([0.91752577, 0.91752577, 0.93877551])}
0.8461538461538461


0.8461538461538461

In [35]:
coder3_specific_d_predictor = predictor.predictor(coder3_specific_d, 'all')
coder3_specific_d_predictor.train_on('d')
coder3_specific_d_predictor.test()

{'fit_time': array([0.53536582, 0.68376088, 0.36124516]), 'score_time': array([0.00112987, 0.00095391, 0.00054097]), 'test_score': array([0.78461538, 0.69230769, 0.76923077]), 'train_score': array([0.96923077, 0.97692308, 0.95384615])}
0.7428571428571429


0.7428571428571429

In [36]:
coder4_specific_d_predictor = predictor.predictor(coder4_specific_d, 'all', k_neighbors= 1)
coder4_specific_d_predictor.train_on('d')
coder4_specific_d_predictor.test()

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 2

In [27]:
Counter(coder4_specific_d.d)

Counter({3.0: 45, 4.0: 15, 2.0: 53, 1.0: 14, 5.0: 1})